# SQL CARTO QUERY FUNCTION
# written for PUI2016_Python2

##### This notebook is set up to link as a default to the fb55 account.  To turn in the homework use the same account you used in the lab (hvt201)  and query the database that you were querying in class (citibike_feb_2015)

In [1]:
SQL_SOURCE = 'https://fb55.carto.com/api/v2/sql?q='
from __future__ import print_function

import urllib2
import urllib
import StringIO
import ast
import pandas as pd

def queryCartoDB(query, format='CSV', source=SQL_SOURCE):
    '''queries carto datasets from a given carto account
    Arguments: 
    query - string: a valid sql query string
    format - outlut format  OPTIONAL (default CSV)
    source - a valid sql api endpoint OPTIONAL (default carto fb55 account)
    Returns:
    the return of the sql query AS A STRING
    NOTES:
    designed for the carto API, tested only with CSV return format'''
    
    data = urllib.urlencode({'format': format, 'q': query})
    try:
        response = urllib2.urlopen(source, data)
    except urllib2.HTTPError, e:
        raise ValueError('\n'.join(ast.literal_eval(e.readline())['error']))
    except Exception:
        raise
    return response.read()

SyntaxError: invalid syntax (<ipython-input-1-73273d8ed468>, line 24)

In [ ]:
test = '''
SELECT *
FROM citibike
'''

In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(test)), sep=',')

# Task 1

In [ ]:
query = '''
SELECT * FROM citibike
WHERE tripduration < 10800
ORDER BY start_station_id, tripduration
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(query)), sep=',')

In [ ]:
query = '''
SELECT * FROM citibike
WHERE tripduration < 10800
ORDER BY start_station_id, tripduration
LIMIT 10
'''
print('TOP 10')
pd.read_csv(StringIO.StringIO(queryCartoDB(query)), sep=',')

In [ ]:
query = '''
SELECT * FROM citibike
WHERE tripduration < 10800
ORDER BY start_station_id DESC, tripduration DESC
LIMIT 10
'''
print('BOTTOM 10')
pd.read_csv(StringIO.StringIO(queryCartoDB(query)), sep=',')

In [ ]:
query = '''
SELECT DISTINCT start_station_id FROM citibike
ORDER BY start_station_id
'''
stations = pd.read_csv(StringIO.StringIO(queryCartoDB(query)), sep=',')
stations = stations['start_station_id'].values
print("Unique Stations:")
for i in range(0, len(stations)):
    print (stations[i])

In [ ]:
query = '''
SELECT COUNT(*) FROM citibike
'''
count = pd.read_csv(StringIO.StringIO(queryCartoDB(query)), sep=',')['count'][0]

query = '''
SELECT AVG(tripduration) FROM citibike
'''
avg = pd.read_csv(StringIO.StringIO(queryCartoDB(query)), sep=',')['avg'][0]

query = '''
SELECT MAX(tripduration) FROM citibike
'''
maxim = pd.read_csv(StringIO.StringIO(queryCartoDB(query)), sep=',')['max'][0]

query = '''
SELECT MIN(tripduration) FROM citibike
'''
minim = pd.read_csv(StringIO.StringIO(queryCartoDB(query)), sep=',')['min'][0]

print('AGGREGATION FUNCTIONS - TRIP DURATION:')
print('Count:', count)
print('Average:', avg)
print('Maximum:', maxim)
print('Minimum:', minim)

# Task 2

In [ ]:
query = '''
SELECT * FROM citibike
WHERE starttime >= '2015-02-02 00:00:00'
AND starttime > '2015-02-03 00:00:00'
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(query)), sep=',')

# Task 3

In [ ]:
query = '''
SELECT
 CDB_TransformToWebmercator(CDB_LatLng(
 start_station_latitude,
 start_station_longitude
 )
 ) as the_geom_webmercator,
 MIN(cartodb_id) as cartodb_id
FROM citibike
GROUP BY
 start_station_latitude,
 start_station_longitude
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(query)), sep=',')

In [ ]:
query = '''
SELECT
 start_station_id,
 CDB_TransformToWebmercator(CDB_LatLng(
 start_station_latitude,
 start_station_longitude
 )
 ) as the_geom_webmercator,
 MIN(cartodb_id) as cartodb_id,
 COUNT(tripduration) as numtrips
FROM citibike
GROUP BY
 start_station_id,
 start_station_latitude,
 start_station_longitude
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(query)), sep=',')

In [ ]:
query = '''
SELECT
 start_station_id,
 CDB_TransformToWebmercator(CDB_LatLng(
 start_station_latitude,
 start_station_longitude
 )
 ) as the_geom_webmercator,
 MIN(cartodb_id) as cartodb_id,
 COUNT(tripduration) as numtrips
FROM citibike
WHERE
ST_DWithin(CDB_LatLng(
 start_station_latitude,
 start_station_longitude
 )::geography,
 CDB_LatLng(40.7577,-73.9857)::geography,
 500)
GROUP BY
 start_station_id,
 start_station_latitude,
 start_station_longitude
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(query)), sep=',')

# Task 4

In [ ]:
query = '''
SELECT
 start_station_id,
 CDB_TransformToWebmercator(CDB_LatLng(start_station_latitude, start_station_longitude)
 ) as the_geom_webmercator,
 MIN(cartodb_id) as cartodb_id,
 AVG(tripduration) as avg_duration
FROM citibike
WHERE
ST_DWithin(
 CDB_LatLng(
 start_station_latitude,
 start_station_longitude
 )::geography,
 CDB_LatLng(40.7577,-73.9857)::geography,
 500)

GROUP BY
 start_station_id,
 start_station_latitude,
 start_station_longitude
ORDER BY avg_duration DESC
LIMIT 1
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(query)), sep=',')

In [ ]:
query = '''
SELECT
 start_station_id,
 end_station_id,
 ST_Makeline(
 CDB_TransformToWebmercator(CDB_LatLng(
 start_station_latitude,
 start_station_longitude
 )),
 CDB_TransformToWebmercator(CDB_LatLng(
 end_station_latitude,
 end_station_longitude
 ))
 ) as the_geom_webmercator,
 MIN(cartodb_id) as cartodb_id,
 AVG(tripduration) as avg_duration
FROM citibike
WHERE
tripduration<7200
and
ST_DWithin(
CDB_LatLng(
 start_station_latitude,
 start_station_longitude
 )::geography,
 CDB_LatLng(40.7577,-73.9857)::geography,
 500)
GROUP BY
 start_station_id,
 start_station_latitude,
 start_station_longitude,
 end_station_id,
 end_station_latitude,
 end_station_longitude
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(query)), sep=',')

# extra credit: make the function python 2 and 3 compatible so that it works on the  PUI2016_Python3 kernel

## ASH - Score 100. EXtra Credit 0